In [ ]:
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class co2TimeSeriesDataset(Dataset):
    def __init__(self, co2_file):
        self.df = pd.read_csv(co2_file)
        self.features = self.df.columns 
        self.df.insert(len(self.features), "target", pd.Series(self.df['co2'][1:].values))
        self.df = self.df.dropna()
        self.size = len(self.df)
        self.data = torch.tensor(np.array(self.df), dtype=torch.float).to(device)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # return (co2_values, co2_next_year)
        return self.data[idx, :, 1:2], self.data[idx, -1, 2:]
    
    def create_window_dataset(self, window_size, starting_idx=0):
        self.data = torch.tensor( np.array([ self.data[starting_idx+idx : starting_idx + idx+window_size].numpy() for idx in range(self.size) if (starting_idx + idx + window_size) <= self.size ]))
    
    def scale_data(self):
        self.scaler = self.scaler.fit(self.data[:,-1].reshape(-1,1))
        self.data[:,-1] = torch.tensor(self.scaler.transform(self.data[:,-1].reshape(-1,1))).reshape(-1)

In [ ]:
def Kfolds(k, dataset):
    size = len(dataset)
    size_fold = [ size // k for x in range(k) ]
    if sum(size_fold) != size:
        size_fold[-1] += size - (size_fold[0]*k)
    kfolds = torch.utils.data.random_split(dataset, [x for x in size_fold])
    return kfolds

def stop_criteria(losses, thr):
    if len(losses) < 2:
        return (False,)
    if statistics.stdev(losses) < thr:
        return (True, statistics.stdev(losses))
    return (False, statistics.stdev(losses))

def combinations_parameters(parameters):
    param = list(parameters.keys())
    res = []
    idx = [0 for x in range(len(param))]
    update_idx = len(idx)-1
    loop = True
    while True:
        if update_idx == -1:
            break
        res.append([ parameters[x][idx[param.index(x)] ] for x in param])
        update_idx = len(idx) -1
        idx[update_idx] += 1
        while idx[update_idx] == len(parameters[param[update_idx]]):
            idx[update_idx] = 0
            update_idx -= 1
            if update_idx == -1:
                break
            idx[update_idx] +=1
    return res
        

In [ ]:
def kFoldCrossValidation(dataset, k, model_name, parameters, stop_size_std):
    
    # create a list with the k folds
    kfolds = Kfolds(k, dataset)
    validation_losses = []
    print(f"Model: {model_name}\nParameters: {parameters}")
    # k-fold cross validation
    for x in range(k):
    
        # model setup
        model_name = model_name.lower()
        if model_name == "rnn":
            model = RNN(parameters["input_size"], parameters["hidden_size"], parameters["num_layers"], parameters["output_size"]).to(device)
        elif model_name == "lstm":
            model = LSTM(parameters["input_size"], parameters["hidden_size"], parameters["num_layers"], parameters["output_size"]).to(device)
        elif model_name == "gru":
            model = GRU(parameters["input_size"], parameters["hidden_size"], parameters["num_layers"], parameters["output_size"]).to(device)
        
        # loss and optimizer selection
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=parameters["learning_rate"])
    
        # kfolds split in test and validation sets
        print(f"Fold {x+1}")
        train_set_k = torch.utils.data.ConcatDataset([kfolds[c] for c in range(k) if c != x])
        train_dataset_loader = DataLoader(train_set_k, batch_size=parameters["batch_size"], shuffle=True)
        validation_dataset_loader = DataLoader(kfolds[x], batch_size=parameters["batch_size"], shuffle=True)
    
        # train until convergence
        losses = []
        epoch = 1
        while True:
        
            loss_batch = []
            for i, (features, labels) in enumerate(train_dataset_loader):
        
                # Forward pass
                outputs = model(features)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_batch.append(loss.item())
                
            # check stopping criterion every epoch
            mean_loss_batch = statistics.mean(loss_batch)
            losses.append(mean_loss_batch)
            stop = stop_criteria(losses, 10*parameters["learning_rate"])
            if stop[0] or epoch==7000:
                break
            if len(losses) == stop_size_std[0]:
                losses = losses[1:]
            if epoch%10 == 0:
                print(f"Epoch {epoch}\nLoss {mean_loss_batch}")
                print(f"stdev: {stop[1]}\n")
            epoch+=1
    
        # perform validation
        with torch.no_grad():
            validation_batch_losses = []
            for features, labels in validation_dataset_loader:
                outputs = model(features)
        
                loss = criterion(outputs,labels)
                validation_batch_losses.append(loss.item())
        
            validation_batch_loss_mean = statistics.mean(validation_batch_losses)
            print(f"Validation Loss: {validation_batch_loss_mean}")
            validation_losses.append(validation_batch_loss_mean)
    return statistics.mean(validation_losses)

In [ ]:
def HyperParametersTuning(dataset, k, model, hyperparam, stop_criteria):
    combinations = combinations_parameters(hyperparam)
    res_tuning = {}
    for comb in combinations:
        temp_hyperparam = {}
        i=0
        for par in hyperparam:
            temp_hyperparam[par] = comb[i]
            i+=1
        mean = kFoldCrossValidation(dataset, k, model, temp_hyperparam, stop_criteria)
        res_tuning[mean] = comb
    best = min(list(res_tuning.keys()))
    return (best, res_tuning[best])
        

In [ ]:
# defining RNN model

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity='relu')
        
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.rand(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.rnn(x, h0)
        
        # take only the last prediction of the sequence
        out = out[:, -1, :]
        
        out = self.fc(out)
        
        return out

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.rand(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.rand(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0,c0))
        
        out = out[:, -1, :]
        
        
        out = self.fc(out)

        return out

In [ ]:
# load dataset, defining sequence length
sequence_length = 10
co2_dataset = co2TimeSeriesDataset("./datasets/co2 dataset/1800_co2.csv")
co2_dataset.create_window_dataset(sequence_length)

In [ ]:
# stopping criteria
size_queue = 10
std_dev = 0.09

# k for kfolds
k = 5

# Hyperparameters for rnn
hyperparam = {"input_size" : [1],
              "output_size" : [1],
              "hidden_size" : [2, 5, 10],
              "num_layers" : [1, 2],
              "batch_size" : [10, 20, 40],
              "learning_rate" : [0.001, 0.01, 0.1]
             }
HyperParametersTuning(co2_dataset, k, "rnn", hyperparam, (size_queue, std_dev))

## LSTM

In [ ]:
# stopping criteria
size_queue = 30
std_dev = 0.1

# k for kfolds
k = 5

# Hyperparameters for rnn
hyperparam = {"input_size" : [1],
              "output_size" : [1],
              "hidden_size" : [10],#[30, 70, 100],
              "num_layers" : [1],
              "batch_size" : [5],
              "learning_rate" : [0.001]#[0.001, 0.01]
             }
HyperParametersTuning(co2_dataset, k, "lstm", hyperparam, (size_queue, std_dev))

In [ ]:
# stopping criteria
size_queue = 10
std_dev = 0.09

# k for kfolds
k = 5

# Hyperparameters for rnn
hyperparam = {"input_size" : [1],
              "output_size" : [1],
              "hidden_size" : [5, 10, 20, 30, 40],
              "num_layers" : [1],
              "batch_size" : [10, 20, 40],
              "learning_rate" : [0.001, 0.01, 0.1]
             }
HyperParametersTuning(co2_dataset, k, "lstm", hyperparam, (size_queue, std_dev))

# Land-ocean temperature anomaly prediction from co2

In [ ]:
class AnomalyTemperaturesTimeSeriesDataset(Dataset):
    def __init__(self, file):
        self.df = pd.read_csv(file)
        self.features = self.df.columns 
        self.df = self.df.dropna()
        self.size = len(self.df)
        self.data = torch.tensor(np.array(self.df), dtype=torch.float).to(device)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx, :, 1:2], self.data[idx, -1, 2:]
    
    def create_window_dataset(self, window_size, starting_idx=0):
        self.data = torch.tensor( np.array([ self.data[starting_idx+idx : starting_idx + idx+window_size].numpy() for idx in range(self.size) if (starting_idx + idx + window_size) <= self.size ]))
    
    def scale_data(self):
        self.scaler = self.scaler.fit(self.data[:,-1].reshape(-1,1))
        self.data[:,-1] = torch.tensor(self.scaler.transform(self.data[:,-1].reshape(-1,1))).reshape(-1)

In [ ]:
# load dataset, defining sequence length
sequence_length = 10
anomaly_temperatures_dataset = AnomalyTemperaturesTimeSeriesDataset("./datasets/land-ocean-temperatures/anomaly_temperatures.csv")
anomaly_temperatures_dataset.create_window_dataset(sequence_length)

## RNN

In [ ]:
# stopping criteria
size_queue = 10
std_dev = 0.09

# k for kfolds
k = 5

# Hyperparameters for rnn
hyperparam = {"input_size" : [1],
              "output_size" : [2],
              "hidden_size" : [2, 5, 10],
              "num_layers" : [1, 2],
              "batch_size" : [20],
              "learning_rate" : [0.001, 0.01, 0.1]
             }
HyperParametersTuning(anomaly_temperatures_dataset, k, "rnn", hyperparam, (size_queue, std_dev))

## LSTM

In [ ]:
# stopping criteria
size_queue = 10
std_dev = 0.09

# k for kfolds
k = 5

# Hyperparameters for lstm
hyperparam = {"input_size" : [1],
              "output_size" : [2],
              "hidden_size" : [5, 10, 20, 30, 40],
              "num_layers" : [1],
              "batch_size" : [10, 20, 40],
              "learning_rate" : [0.001, 0.01, 0.1]
             }
HyperParametersTuning(anomaly_temperatures_dataset, k, "lstm", hyperparam, (size_queue, std_dev))

# Mortality, sea level and wildfires prediction from anomaly temperatures of land and oceans

## Mortality

In [ ]:
class MortalityTimeSeriesDataset(Dataset):
    def __init__(self, file):
        self.df = pd.read_csv(file)
        self.features = self.df.columns 
        self.df.insert(loc=3, column="mortality_feature", value=self.df['mortality_x_100k'].shift(1))
        self.df = self.df.dropna()
        
        self.size = len(self.df)
        self.data = torch.tensor(np.array(self.df), dtype=torch.float).to(device)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx, :, 1:4], self.data[idx, -1, 4:]
    
    def create_window_dataset(self, window_size, starting_idx=0):
        self.data = torch.tensor( np.array([ self.data[starting_idx+idx : starting_idx + idx+window_size].numpy() for idx in range(self.size) if (starting_idx + idx + window_size) <= self.size ]))
    
    def scale_data(self):
        self.scaler = self.scaler.fit(self.data[:,-1].reshape(-1,1))
        self.data[:,-1] = torch.tensor(self.scaler.transform(self.data[:,-1].reshape(-1,1))).reshape(-1)

In [ ]:
# load dataset, defining sequence length
sequence_length = 10
mortality_usa_dataset = MortalityTimeSeriesDataset("./datasets/mortality dataset/temp_death_usa.csv")
mortality_usa_dataset.create_window_dataset(sequence_length)

FAILED

## Sea Level

In [ ]:
class SeaLevelTimeSeriesDataset(Dataset):
    def __init__(self, file):
        self.df = pd.read_csv(file)
        self.features = self.df.columns 
        self.df.insert(loc=3, column="GMSL_feature", value=self.df['GMSL'].shift(1))
        self.df = self.df.dropna()
        
        self.size = len(self.df)
        self.data = torch.tensor(np.array(self.df), dtype=torch.float).to(device)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx, :, 1:4], self.data[idx, -1, 4:]
    
    def create_window_dataset(self, window_size, starting_idx=0):
        self.data = torch.tensor( np.array([ self.data[starting_idx+idx : starting_idx + idx+window_size].numpy() for idx in range(self.size) if (starting_idx + idx + window_size) <= self.size ]))
    
    def scale_data(self):
        self.scaler = self.scaler.fit(self.data[:,-1].reshape(-1,1))
        self.data[:,-1] = torch.tensor(self.scaler.transform(self.data[:,-1].reshape(-1,1))).reshape(-1)

In [ ]:
# load dataset, defining sequence length
sequence_length = 5
sea_level_dataset = SeaLevelTimeSeriesDataset("./datasets/sea level dataset/clean_sea_levels.csv")
sea_level_dataset.create_window_dataset(sequence_length)

In [ ]:
# stopping criteria
size_queue = 10
std_dev = 0.09

# k for kfolds
k = 5

# Hyperparameters for rnn
hyperparam = {"input_size" : [3],
              "output_size" : [1],
              "hidden_size" : [2, 5, 10, 20],
              "num_layers" : [1, 2],
              "batch_size" : [10],
              "learning_rate" : [0.001, 0.01, 0.1]
             }
HyperParametersTuning(sea_level_dataset, k, "rnn", hyperparam, (size_queue, std_dev))

In [ ]:
# stopping criteria
size_queue = 10
std_dev = 0.09

# k for kfolds
k = 5

# Hyperparameters for rnn
hyperparam = {"input_size" : [3],
              "output_size" : [1],
              "hidden_size" : [40, 70],
              "num_layers" : [1],
              "batch_size" : [10, 20, 40],
              "learning_rate" : [0.001, 0.01, 0.1]
             }
HyperParametersTuning(sea_level_dataset, k, "lstm", hyperparam, (size_queue, std_dev))

# CO2 model prediction training

In [ ]:
        train_size = 0.8
        train_size = int(train_size * len(co2_dataset))
        train_set, test_set = torch.utils.data.random_split(co2_dataset, [train_size, len(co2_dataset) - train_size])
        train_set_loader = DataLoader(train_set, batch_size=40, shuffle=True)
        test_set_loader = DataLoader(test_set, batch_size=40, shuffle=True)
        co2_rnn_model = RNN(1, 10, 2, 1).to(device)
        
        # loss and optimizer selection
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(co2_rnn_model.parameters(), lr=0.01)
        
        # train until convergence
        losses = []
        history_loss = []
        epoch = 1
        while True:
        
            loss_batch = []
            for i, (features, labels) in enumerate(train_set_loader):
        
                # Forward pass
                outputs = co2_rnn_model(features)
                loss = criterion(outputs, labels)
 
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_batch.append(loss.item())
                
            # check stopping criterion every epoch
            mean_loss_batch = statistics.mean(loss_batch)
            losses.append(mean_loss_batch)
            history_loss.append(mean_loss_batch)
            stop = stop_criteria(losses, 0.01)
            if stop[0] or epoch==3000:
                break
            if len(losses) == 10:
                losses = losses[1:]
            if epoch%10 == 0:
                print(f"Epoch {epoch}\nLoss {mean_loss_batch}")
                print(f"stdev: {stop[1]}\n")
            epoch+=1
    
        # perform test
        with torch.no_grad():
            validation_batch_losses = []
            for features, labels in test_set_loader:
                outputs = co2_rnn_model(features)
        
                loss = criterion(outputs,labels)
                validation_batch_losses.append(loss.item())
        
            validation_batch_loss_mean = statistics.mean(validation_batch_losses)
            print(f"Test Loss: {validation_batch_loss_mean}")

# Anomaly temperatures prediction model training

In [ ]:
        dataset = anomaly_temperatures_dataset
        batch_size = 10
        train_size = 0.8
        train_size = int(train_size * len(dataset))
        train_set, test_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
        train_set_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        test_set_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)
        temp_anomaly_lstm_model = LSTM(1, 40, 1, 2).to(device)
        
        # loss and optimizer selection
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(temp_anomaly_lstm_model.parameters(), lr=0.001)
        
        # train until convergence
        losses = []
        history_loss = []
        epoch = 1
        while True:
        
            loss_batch = []
            for i, (features, labels) in enumerate(train_set_loader):
        
                # Forward pass
                outputs = temp_anomaly_lstm_model(features)
                loss = criterion(outputs, labels)
 
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_batch.append(loss.item())
                
            # check stopping criterion every epoch
            mean_loss_batch = statistics.mean(loss_batch)
            losses.append(mean_loss_batch)
            history_loss.append(mean_loss_batch)
            stop = stop_criteria(losses, 0.0005)
            if stop[0] or epoch==5000:
                break
            if len(losses) == 10:
                losses = losses[1:]
            if epoch%10 == 0:
                print(f"Epoch {epoch}\nLoss {mean_loss_batch}")
                print(f"stdev: {stop[1]}\n")
            epoch+=1
    
        # perform validation
        with torch.no_grad():
            validation_batch_losses = []
            for features, labels in test_set_loader:
                outputs = temp_anomaly_lstm_model(features)
        
                loss = criterion(outputs,labels)
                validation_batch_losses.append(loss.item())
        
            validation_batch_loss_mean = statistics.mean(validation_batch_losses)
            print(f"Test Loss: {validation_batch_loss_mean}")

# Sea level future predictions model

In [ ]:
        dataset = sea_level_dataset
        batch_size = 10
        train_size = 0.8
        train_size = int(train_size * len(dataset))
        train_set, test_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
        train_set_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        test_set_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)
        sea_level_lstm_model = LSTM(3, 40, 1, 1).to(device)
        
        # loss and optimizer selection
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(sea_level_lstm_model.parameters(), lr=0.001)
        
        # train until convergence
        losses = []
        history_loss = []
        epoch = 1
        while True:
        
            loss_batch = []
            for i, (features, labels) in enumerate(train_set_loader):
        
                # Forward pass
                outputs = sea_level_lstm_model(features)
                loss = criterion(outputs, labels)
 
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_batch.append(loss.item())
                
            # check stopping criterion every epoch
            mean_loss_batch = statistics.mean(loss_batch)
            losses.append(mean_loss_batch)
            history_loss.append(mean_loss_batch)
            stop = stop_criteria(losses, 0.0005)
            if stop[0] or epoch==50000:
                break
            if len(losses) == 10:
                losses = losses[1:]
            if epoch%10 == 0:
                print(f"Epoch {epoch}\nLoss {mean_loss_batch}")
                print(f"stdev: {stop[1]}\n")
            epoch+=1
    
        # perform validation
        with torch.no_grad():
            validation_batch_losses = []
            for features, labels in test_set_loader:
                outputs = temp_anomaly_lstm_model(features)
        
                loss = criterion(outputs,labels)
                validation_batch_losses.append(loss.item())
        
            validation_batch_loss_mean = statistics.mean(validation_batch_losses)
            print(f"Test Loss: {validation_batch_loss_mean}")

# Future predictions with the models

In [ ]:
year = 20
time_series = co2_dataset[-1][0].unsqueeze(0).detach().clone()
input_sea_level = sea_level_dataset[-1][0][-1:,-3:].unsqueeze(0)
co2_future_values = []
temp_future_land = []
temp_future_ocean = []
sea_level_future = []


for x in range(year):
    temp_prediction_co2 = co2_rnn_model(time_series)
    co2_future_values.append(float(temp_prediction_co2[0][0]))
    time_series = torch.cat((time_series, temp_prediction_co2.unsqueeze(0)),1)
    time_series = time_series[:,1:,:]
    
    # prediction temperature anomalies
    temp_prediction_anomaly = temp_anomaly_lstm_model(time_series[:,:10,:])
    time_series = time_series[:,1:,:]
    temp_future_land.append(float(temp_prediction_anomaly[0][0]))
    temp_future_ocean.append(float(temp_prediction_anomaly[0][1]))
    temp_prediction_sea_level = sea_level_lstm_model(input_sea_level)
    sea_level_future.append(float(temp_prediction_sea_level[0][0]))
    v = torch.cat((temp_prediction_anomaly, temp_prediction_sea_level), 1)
    input_sea_level = torch.cat((input_sea_level, v.unsqueeze(0)), 1)